## Deliverable 2. Create a Customer Travel Destinations Map.

In [23]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [24]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Rikitea,PF,2020-08-18 17:04:41,-23.12,-134.97,71.65,81,3,12.08
1,1,Busselton,AU,2020-08-18 17:04:41,-33.65,115.33,45.00,72,74,5.14
2,2,Luderitz,NaN,2020-08-18 17:04:41,-26.65,15.16,62.20,32,0,29.53
3,3,Saskylakh,RU,2020-08-18 17:01:58,71.92,114.08,46.45,85,100,8.39
4,4,Mataura,NZ,2020-08-18 17:04:42,-46.19,168.86,43.68,76,100,4.34
5,5,Kodiak,US,2020-08-18 17:04:42,57.79,-152.41,57.20,82,90,1.95
6,6,Carnarvon,AU,2020-08-18 17:04:42,-24.87,113.63,62.60,88,53,10.29
7,7,Camacha,PT,2020-08-18 17:04:42,33.08,-16.33,77.00,65,20,12.75
8,8,Vaini,TO,2020-08-18 17:04:43,-21.20,-175.20,73.40,94,90,12.75
9,9,Barrow,US,2020-08-18 17:04:43,71.29,-156.79,42.80,100,90,17.22


In [25]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
Maximum_Temp = float(input("What is the maximum temperature you would like for your vacation?"))
Minimum_Temp = float(input("What is the minimum temperature you would like for your vacation?"))

What is the maximum temperature you would like for your vacation?77
What is the minimum temperature you would like for your vacation?35


In [26]:
vacation_df = city_data_df.loc[(city_data_df["Max Temp"] < Maximum_Temp) & (city_data_df["Max Temp"] > Minimum_Temp)]
vacation_df.head(10)

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Rikitea,PF,2020-08-18 17:04:41,-23.12,-134.97,71.65,81,3,12.08
1,1,Busselton,AU,2020-08-18 17:04:41,-33.65,115.33,45.00,72,74,5.14
2,2,Luderitz,NaN,2020-08-18 17:04:41,-26.65,15.16,62.20,32,0,29.53
3,3,Saskylakh,RU,2020-08-18 17:01:58,71.92,114.08,46.45,85,100,8.39
4,4,Mataura,NZ,2020-08-18 17:04:42,-46.19,168.86,43.68,76,100,4.34
5,5,Kodiak,US,2020-08-18 17:04:42,57.79,-152.41,57.20,82,90,1.95
6,6,Carnarvon,AU,2020-08-18 17:04:42,-24.87,113.63,62.60,88,53,10.29
8,8,Vaini,TO,2020-08-18 17:04:43,-21.20,-175.20,73.40,94,90,12.75
9,9,Barrow,US,2020-08-18 17:04:43,71.29,-156.79,42.80,100,90,17.22
10,10,Gisborne,NZ,2020-08-18 17:01:56,-38.65,178.00,48.99,80,100,1.01


In [27]:
# 4a. Determine if there are any empty rows.
vacation_df.count()

City_ID       408
City          408
Country       401
Date          408
Lat           408
Lng           408
Max Temp      408
Humidity      408
Cloudiness    408
Wind Speed    408
dtype: int64

In [28]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows
clean_df = vacation_df.dropna()

clean_df.count()


City_ID       401
City          401
Country       401
Date          401
Lat           401
Lng           401
Max Temp      401
Humidity      401
Cloudiness    401
Wind Speed    401
dtype: int64

In [29]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Rikitea,PF,71.65,-23.12,-134.97,
1,Busselton,AU,45.00,-33.65,115.33,
3,Saskylakh,RU,46.45,71.92,114.08,
4,Mataura,NZ,43.68,-46.19,168.86,
5,Kodiak,US,57.20,57.79,-152.41,
6,Carnarvon,AU,62.60,-24.87,113.63,
8,Vaini,TO,73.40,-21.20,-175.20,
9,Barrow,US,42.80,71.29,-156.79,
10,Gisborne,NZ,48.99,-38.65,178.00,
11,Punta Arenas,CL,35.60,-53.15,-70.92,


In [30]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index,row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    params["location"] = f"{latitude},{longitude}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
            print("Hotel not found... skipping")
            
hotel_df.head(10)
        

Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... skipping
Hotel not found... s

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Rikitea,PF,71.65,-23.12,-134.97,Pension Maro'i
1,Busselton,AU,45.00,-33.65,115.33,Observatory Guest House
3,Saskylakh,RU,46.45,71.92,114.08,
4,Mataura,NZ,43.68,-46.19,168.86,Ellie's Villa
5,Kodiak,US,57.20,57.79,-152.41,Best Western Kodiak Inn and Convention Center
6,Carnarvon,AU,62.60,-24.87,113.63,Hospitality Carnarvon
8,Vaini,TO,73.40,-21.20,-175.20,Keleti Beach Resort
9,Barrow,US,42.80,71.29,-156.79,King Eider Inn
10,Gisborne,NZ,48.99,-38.65,178.00,ASURE Alfresco Motor Lodge
11,Punta Arenas,CL,35.60,-53.15,-70.92,Hotel Dreams Del Estrecho


In [31]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Rikitea,PF,71.65,-23.12,-134.97,Pension Maro'i
1,Busselton,AU,45.00,-33.65,115.33,Observatory Guest House
4,Mataura,NZ,43.68,-46.19,168.86,Ellie's Villa
5,Kodiak,US,57.20,57.79,-152.41,Best Western Kodiak Inn and Convention Center
6,Carnarvon,AU,62.60,-24.87,113.63,Hospitality Carnarvon
8,Vaini,TO,73.40,-21.20,-175.20,Keleti Beach Resort
9,Barrow,US,42.80,71.29,-156.79,King Eider Inn
10,Gisborne,NZ,48.99,-38.65,178.00,ASURE Alfresco Motor Lodge
11,Punta Arenas,CL,35.60,-53.15,-70.92,Hotel Dreams Del Estrecho
13,Jamestown,US,72.00,42.10,-79.24,Hampton Inn & Suites Jamestown


In [32]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_Vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [33]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]



In [20]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))